# CSE 4633/6633 HW 1

The assignment for this notebook is to implement local search algorithms such as hill-climbing, random restart, and simulated annealing to perform function optimization.

In [5]:
# import required libraries
import time
import random
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [6]:
# Create a complex objective function that we need to maximize.
# Here, the objective function is simply defined as an equation but
# more generally, it could represent natural phenomena, physical laws, or mathematical models
#   X: a 2-dimensional floating-point vector consisting of an x-value and a y-value
#   returns: a scalar floating-point value representing the output of the objective function
def objective_function(X):
    x = X[0]
    y = X[1]
    value = 3 * (1 - x) ** 2 * math.exp(-x ** 2 - (y + 1) ** 2) - 10 * (x / 5 - x ** 3 - y ** 5) * math.exp(
        -x ** 2 - y ** 2) - (1 / 3) * math.exp(-(x + 1) ** 2 - y ** 2)
    return value

# to limit the search space for this problem, we will only consider solutions
# where x ranges from -4 to 4 and y ranges from -4 to 4
lower_bounds = [-4, -4]
upper_bounds = [4, 4]

A 3D plot of the objective function can be viewed [here](https://c3d.libretexts.org/CalcPlot3D/index.html?type=z;z=3(1-x)%5E2*exp(-x%5E2-(y+1)%5E2)-10(x/5-x%5E3-y%5E5)*exp(-x%5E2-y%5E2)-(1/3)*exp(-(x+1)%5E2-y%5E2);visible=true;umin=-4;umax=4;vmin=-4;vmax=4;grid=30;format=normal;alpha=-1;hidemyedges=true;constcol=rgb(255,0,0);view=0;contourcolor=red;fixdomain=false;contourplot=true;showcontourplot=false;firstvalue=-1;stepsize=0.2;numlevels=11;list=;uselist=false;xnum=46;ynum=46;show2d=false;hidesurface=false;hidelabels=true;showprojections=false;surfacecontours=true;projectioncolor=rgba(255,0,0,1);showxygrid=false;showxygridonbox=false;showconstraint=false&type=window;hsrmode=3;nomidpts=true;anaglyph=-1;center=-5.2487653277286155,6.815843602039553,5.098503557610455,1;focus=0,0,0,1;up=0.39284920127083023,-0.3373981166615778,0.8554718089651412,1;transparent=false;alpha=140;twoviews=false;unlinkviews=false;axisextension=0.7;xaxislabel=x;yaxislabel=y;zaxislabel=z;edgeson=true;faceson=true;showbox=true;showaxes=true;showticks=true;perspective=true;centerxpercent=0.5;centerypercent=0.5;rotationsteps=30;autospin=true;xygrid=false;yzgrid=false;xzgrid=false;gridsonbox=true;gridplanes=false;gridcolor=rgb(128,128,128);xmin=-4;xmax=4;ymin=-4;ymax=4;zmin=-4;zmax=4;xscale=2;yscale=2;zscale=2;zcmin=-8;zcmax=8;xscalefactor=1;yscalefactor=1;zscalefactor=1;tracemode=0;keep2d=false;zoom=0.89)

In [7]:
# Let's try to randomly generate several random inputs to the objective function
# and manually observe how the outputs change with different inputs

X = np.random.uniform(lower_bounds, upper_bounds)
value = objective_function(X)
print('objective_function(%.3f, %.3f) = %.3f' % (X[0], X[1], value))

objective_function(0.593, -3.881) = -0.002


# Part A: Hill-climbing

In [10]:
# TODO: Hill-climbing search algorithm: a loop that continually moves in the direction of increasing value.
# It terminates when it reaches a “peak” where no neighbor has a higher value.
#   objective function: function to be maximized
#   initial_state: initial (x, y) vector
#   step_size: numerical interval by which to change the current (x,y) state to generate a new neighboring state
#   print_iters: set to True to print out the current value at each iteration
#   returns: best [x, y] solution found
def hill_climbing(objective_function, initial_state = np.array([0,0]), step_size = 0.01, print_iters=True):
  # set the starting point of the search algorithm
  current_state = initial_state
  # loop until a peak is found
  i = 0
  while True:
    # Step 1: create a list of neighboring states to the current state
    neighbors = []
    for dx in [-step_size, 0, step_size]:
      for dy in [-step_size, 0, step_size]:
        # skip the current state itself
        if dx == 0 and dy == 0:
          continue
        # create a new state by adding the increments to the current state
        neighbor = current_state + np.array([dx, dy])
        # check if the new state is within the bounds
        if lower_bounds[0] <= neighbor[0] <= upper_bounds[0] and lower_bounds[1] <= neighbor[1] <= upper_bounds[1]:
          # add the new state to the list of neighbors
          neighbors.append(neighbor)
    # Step 2: calculate the objective function at each of the neighboring states
    values = [objective_function(neighbor) for neighbor in neighbors]
    # Step 3: determine the highest-valued neighboring state
    best_index = np.argmax(values)
    best_neighbor = neighbors[best_index]
    best_value = values[best_index]
    # Step 4: compare the highest value among neighboring states to the current value
    #         if the latter is higher, we have found a peak -> return the current state
    #         if the former is higher, assign current state to be the best neighbor state
    current_value = objective_function(current_state)
    if best_value <= current_value:
      return current_state
    else:
      current_state = best_neighbor
    if print_iters:
      print(' Iteration: {}, current_state: {}, current_value:{}'.format(i, current_state, current_value))
    i += 1


In [11]:
hill_climbing_solution = hill_climbing(objective_function)
print('Hill climbing solution is:', hill_climbing_solution)

 Iteration: 0, current_state: [-0.01 -0.01], current_value:0.9810118431238463
 Iteration: 1, current_state: [-0.02 -0.02], current_value:1.043242185894391
 Iteration: 2, current_state: [-0.03 -0.03], current_value:1.106488736553738
 Iteration: 3, current_state: [-0.04 -0.04], current_value:1.1706625998511844
 Iteration: 4, current_state: [-0.05 -0.05], current_value:1.2356733918938736
 Iteration: 5, current_state: [-0.06 -0.06], current_value:1.3014293628501752
 Iteration: 6, current_state: [-0.07 -0.07], current_value:1.3678375223821952
 Iteration: 7, current_state: [-0.08 -0.08], current_value:1.4348037678719663
 Iteration: 8, current_state: [-0.09 -0.09], current_value:1.5022330154933727
 Iteration: 9, current_state: [-0.1 -0.1], current_value:1.5700293341667775
 Iteration: 10, current_state: [-0.11 -0.11], current_value:1.6380960824157307
 Iteration: 11, current_state: [-0.12 -0.12], current_value:1.7063360481252672
 Iteration: 12, current_state: [-0.13 -0.13], current_value:1.7746

# Part B: Random Restart

In [23]:
# TODO: Improvement to the Hill-climbing search algorithm using random restarts
#   objective function: function to be maximized
#   lower_bounds: minimum allowable values for the input vector to the objective function 
#   upper_bounds: maximum allowable values for the input vector to the objective function
#   step_size: numerical interval by which to change the current (x,y) state to generate a new neighboring state
#   num_restarts: how many times to restart hill-climbing
#   returns: best [x, y] solution found
def random_restart_hill_climbing(objective_function, lower_bounds, upper_bounds, step_size = 0.001, num_restarts=30):
  best_state = None
  best_value = -float('inf')
  # loop for a given number of restarts
  for i in range(num_restarts):
    # generate a random initial state within the bounds
    initial_state = np.random.uniform(lower_bounds, upper_bounds)
    # perform hill climbing from the initial state
    current_state = hill_climbing(objective_function, initial_state, step_size, print_iters=False)
    # evaluate the current state using the objective function
    current_value = objective_function(current_state)
    # compare the current value with the best value so far
    if current_value > best_value:
      # update the best state and value
      best_state = current_state
      best_value = current_value
    # print the iteration number and state
    print(f" Iteration {i+1}: {current_state}")
  # return the best state found
  return best_state

In [24]:
random_restart_solution = random_restart_hill_climbing(objective_function, lower_bounds, upper_bounds)
print('Random restart hill climbing solution is:', random_restart_solution)

 Iteration 1: [ 1.28532791 -0.00474635]
 Iteration 2: [-0.00891919  1.58131696]
 Iteration 3: [ 1.28545443 -0.00510339]
 Iteration 4: [-0.00897161  1.58096791]
 Iteration 5: [ 1.28605579 -0.00489621]
 Iteration 6: [-0.46030551 -0.62941801]
 Iteration 7: [ 1.28534061 -0.0046979 ]
 Iteration 8: [ 1.28541879 -0.00491017]
 Iteration 9: [-0.45970625 -0.62916621]
 Iteration 10: [-0.46041796 -0.62953762]
 Iteration 11: [-0.45969229 -0.6290614 ]
 Iteration 12: [-0.00888894  1.58122717]
 Iteration 13: [ 1.28564869 -0.00489413]
 Iteration 14: [-0.45980486 -0.62872495]
 Iteration 15: [ 1.28609589 -0.00468792]
 Iteration 16: [-0.0091231   1.58177558]
 Iteration 17: [ 1.28546776 -0.0045917 ]
 Iteration 18: [ 1.28579148 -0.00523232]
 Iteration 19: [ 1.28573607 -0.00457228]
 Iteration 20: [-0.45979301 -0.62867336]
 Iteration 21: [ 1.28577287 -0.00436941]
 Iteration 22: [-0.00919693  1.58173595]
 Iteration 23: [ 1.2857483  -0.00440488]


KeyboardInterrupt: ignored

# Part C: Simulated Annealing

In [14]:
# TODO: Simulated annealing algorithm
#   objective function: function to be maximized
#   lower_bounds: minimum allowable values for the input vector to the objective function 
#   upper_bounds: maximum allowable values for the input vector to the objective function
#   returns: best [x, y] solution found

def simulated_annealing(objective_function, lower_bounds, upper_bounds):
  # initialize the temperature and cooling rate
  step_size = 0.1
  temperature = 1000
  cooling_rate = 0.99
  # generate a random initial state within the bounds
  current_state = np.random.uniform(lower_bounds, upper_bounds)
  # evaluate the current state using the objective function
  current_value = objective_function(current_state)
  # initialize the best state and value
  best_state = current_state
  best_value = current_value
  iteration = 0 
  # loop until the temperature reaches zero or close to zero
  while temperature > 1e-3:
    # generate a random neighbor state within the bounds by perturbing the current state slightly
    neighbor_state = current_state + np.random.normal(0, step_size)
    neighbor_state = np.clip(neighbor_state, lower_bounds, upper_bounds)
    # evaluate the neighbor state using the objective function
    neighbor_value = objective_function(neighbor_state)
    # calculate the difference between the neighbor value and the current value (delta E in pseudocode)
    delta = neighbor_value - current_value
    # if the neighbor value is better than or equal to the current value, accept it as the new current state (always accept uphill moves in pseudocode)
    if delta >= 0:
      current_state = neighbor_state
      current_value = neighbor_value 
    else:
      probability = np.exp(delta / temperature) 
      if np.random.rand() < probability:
        current_state = neighbor_state 
        current_value = neighbor_value 
    # update the best state and value if needed 
    if current_value > best_value:
      best_state = current_state 
      best_value = current_value 
    # decrease temperature according to cooling schedule (linearly decreasing function in this example) (decrease T according to schedule in pseudocode)
    temperature -= cooling_rate 
    
    print(f"Iteration {iteration}: Current state = {current_state}, Current value = {current_value}")
    iteration +=1
  return best_state



In [15]:
simulated_annealing_solution = simulated_annealing(objective_function, lower_bounds, upper_bounds)
print('Simulated annealing solution is:', simulated_annealing_solution)

Iteration 0: Current state = [2.07358411 1.99163531], Current value = 0.10239555616775164
Iteration 1: Current state = [2.01908131 1.93713251], Current value = 0.13970813587161982
Iteration 2: Current state = [1.95096305 1.86901425], Current value = 0.2017180612853735
Iteration 3: Current state = [2.00500337 1.92305457], Current value = 0.15101665801362796
Iteration 4: Current state = [2.0001575  1.91820869], Current value = 0.15508156861812344
Iteration 5: Current state = [2.11891224 2.03696344], Current value = 0.07819118430594066
Iteration 6: Current state = [2.09464196 2.01269315], Current value = 0.09045251834734491
Iteration 7: Current state = [2.03826004 1.95631124], Current value = 0.1254511484918943
Iteration 8: Current state = [1.90494696 1.82299815], Current value = 0.2550996870368456
Iteration 9: Current state = [1.8433364 1.7613876], Current value = 0.34342034061295484
Iteration 10: Current state = [1.81864602 1.73669722], Current value = 0.3847406002650028
Iteration 11: C